# LVV-T297: Absolute Astrometric Performance

**Written By: Bryce Kalmbach**

**Last updated: 07-10-2019**

**Tested on Stack Version: w_2019_27**

## Requirements:

[OSS-REQ-0388](https://docushare.lsst.org/docushare/dsweb/Get/LSE-030#page=68)

Median error in absolute position for each axis, RA and DEC, shall be less than 50 milliarcseconds.

## Proposed Test Case:

1. Take images from region overlapping the Gaia footprint.  Repeat at multiple airmasses.

2. Perform source detection and astrometric measurement on images from step 1

3. Cross-match catalog from step 2 with Gaia catalog.  Select sources that are consistent with zero proper motion (according to Gaia).

4. Verify that the median error of the LSST positions (relative to the Gaia positions) is **50 milliarcseconds in RA, Dec independently**

### Import necessary tools

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.persistence import Butler
import lsst.daf.persistence as daf_persistence

from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
# Make our plots nice and readable
plt.rcParams.update({'font.size': 18})

### Identify HSC Data to use

We want to get data from a single visit for this requirement so we choose a visit from the HSC Wide dataset. https://hsc-release.mtk.nao.ac.jp/doc/index.php/database/ has info 
on which tracts are included in the Wide data. We randomly choose tract 9348 for testing.

In [ ]:
# Load a butler for the HSC Wide data
depth = 'WIDE'
butler = daf_persistence.Butler('/datasets/hsc/repo/rerun/DM-13666/%s/'%(depth))

In [ ]:
# Find a visit in the WIDE data for HSC-R band in the tract 9348
! ls /datasets/hsc/repo/rerun/DM-13666/WIDE/deepCoadd/HSC-R/9348/0,0

In [ ]:
# Use visit 23740
band = 'HSC-R'
visit = 23740

In [ ]:
subset = butler.subset('src', filter=band, visit=visit)

In [ ]:
# Load in sources from visit making exceptions for bad ccd 9 and focusing ccds.
hsc_sources_df = None
ccd_lims = []
for dataId in subset.cache:
    try:
        src_cat = butler.get('src', dataId=dataId)
        src_cat_df = src_cat.asAstropy().to_pandas()
        if hsc_sources_df is None:
            hsc_sources_df = pd.DataFrame([], columns=src_cat_df.columns)
            #hsc_sources_df = pd.concat([hsc_sources_df, src_cat_df], sort=False)
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        else:
            #hsc_sources_df = pd.concat([hsc_sources_df, src_cat_df], sort=False)
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        ccd_lims.append([np.degrees(np.max(src_cat_df['coord_ra'])),
                         np.degrees(np.min(src_cat_df['coord_ra'])),
                         np.degrees(np.max(src_cat_df['coord_dec'])),
                         np.degrees(np.min(src_cat_df['coord_dec']))])
    except daf_persistence.butlerExceptions.NoResults as inst:
        print(dataId['ccd'])

In [ ]:
# Total number of HSC Sources
len(hsc_sources_df)

In [ ]:
hsc_sources_coords = SkyCoord(hsc_sources_df['coord_ra']*u.rad, hsc_sources_df['coord_dec']*u.rad)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(hsc_sources_coords.ra.deg, hsc_sources_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('HSC Sources in Visit %i' %visit)

### Load Gaia data

We have previously created a pandas dataframe with Gaia data that overlaps the HSC Wide data footprint. Here we load it in and select the data in the region of the visit.

In [ ]:
# Load in cached gaia data
gaia_df = pd.read_pickle('/project/danielsf/gaia_hsc_overlap_pandas.pickle')

In [ ]:
gaia_df.head()

In [ ]:
# Only select data that falls in the bounds of the HSC CCDs
gaia_visit_df = pd.DataFrame([], columns=gaia_df.columns)
for ccd_corners in ccd_lims:
    gaia_visit_df = gaia_visit_df.append(gaia_df.query('ra < %f and ra > %f and dec < %f and dec > %f' % (ccd_corners[0], ccd_corners[1],
                                                                                                          ccd_corners[2], ccd_corners[3])))

In [ ]:
gaia_coords = SkyCoord(gaia_visit_df['ra']*u.deg, gaia_visit_df['dec']*u.deg)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(gaia_coords.ra.deg, gaia_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('Gaia Sources in Visit %i' %visit)

### Use astropy to match

We will use the `match_to_catalog_sky` method from astropy to do the catalog match.

In [ ]:
idx, sep2d, sep3d = gaia_coords.match_to_catalog_sky(hsc_sources_coords)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(gaia_coords.ra.deg, gaia_coords.dec.deg, c=sep2d.arcsec, s=20, vmax=50)
cb = plt.colorbar()
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
cb.set_label('Distance to match (arcsec)')

In [ ]:
# Look at overall separation
fig = plt.figure(figsize=(10, 8))
n, bins, _ = plt.hist(sep2d.arcsec, label='Gaia Objects', range=(0, 0.1))
plt.axvline(np.median(sep2d.arcsec), 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*np.median(sep2d.arcsec)))
plt.title('Test of Absolute Astrometry')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

In [ ]:
# Requirement specifies looking at RA and dec inpendently
sep_ra = gaia_coords.ra.arcsec - hsc_sources_coords.ra.arcsec[idx]
sep_dec = gaia_coords.dec.arcsec - hsc_sources_coords.dec.arcsec[idx]

In [ ]:
# Requirement specifies RA, dec individually so we look at those here
fig = plt.figure(figsize=(20, 8))

fig.add_subplot(1,2,1)
n, bins, _ = plt.hist(np.abs(sep_ra), label='Gaia Objects', range=(0, 0.1))
plt.axvline(np.median(np.abs(sep_ra)), 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*np.median(np.abs(sep_ra))))
plt.axvline(.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='k', lw=4)
plt.title('Test of Absolute Astrometry RA')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

fig.add_subplot(1,2,2)
n, bins, _ = plt.hist(np.abs(sep_dec), label='Gaia Objects', range=(0, 0.1))
plt.axvline(np.median(np.abs(sep_dec)), 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*np.median(np.abs(sep_dec))))
plt.axvline(0.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='k', lw=4)
plt.title('Test of Absolute Astrometry dec')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

The requirements are satisfied if both RA and dec median values are less than 50 milliarcseconds.